<a href="https://colab.research.google.com/github/BaronVonBussin/NewTransit/blob/main/20150115_newr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
import os
import sys
import pandas as pd
import numpy as np
import math
import time
import logging
from datetime import datetime
from typing import Dict, Tuple, Optional, List, Any
from tqdm import tqdm

# Constants for configuration
VALID_CHILD_PERIODS = ['D', 'W', 'M', 'Q']
VALID_PARENT_PERIODS = ['W', 'M', 'Q', 'Y']
DEFAULT_JOBNAME = 'gelset_20250107'

class GelSetProcessor:
    def __init__(self,
                 input_dir="/content/input",
                 output_child_dir="/content/output_child",
                 output_parent_dir="/content/output_parent",
                 test_dir="/content/test",
                 child_period="D",
                 parent_period="M",
                 jobname="gelset_20250107"):
        self.input_dir = input_dir
        self.output_child_dir = output_child_dir
        self.output_parent_dir = output_parent_dir
        self.test_dir = test_dir
        self.child_period = child_period
        self.parent_period = parent_period
        self.jobname = jobname

        #print("✅ GelSetProcessor initialized!")
        #print(f"📂 Input Directory: {self.input_dir}")
        #print(f"📂 Output Child Directory: {self.output_child_dir}")
        #print(f"📂 Output Parent Directory: {self.output_parent_dir}")
        #print(f"📂 Test Directory: {self.test_dir}")

        self.export_columns = [
            # Input fields
            'ticker', 'date', 'parent', 'sequence', 'open', 'high', 'low', 'close', 'volume',

            # Prior Child (PC) fields
            'bpb_reu_value', 'bpb_red_value', 'bpb_reu_flag', 'bpb_red_flag',
            'bpb_re_flag', 'bpb_twoway_flag',
            'bpb_pripr', 'bpb_ce_percent',
            'bpb_epc', 'bpb_epc_dir',
            'bpb_e1_value', 'bpb_e2_value',

             # Intra-Parent (IP) fields
            'gel_o', 'gel_h', 'gel_l',
            'gel_reu_value', 'gel_red_value', 'gel_reu_flag', 'gel_red_flag',
            'ip_re_flag', 'ip_twoway_flag', 'ip_rpc_this_row',
            'gel_pripr', 'ip_ce_percent',
            'ip_epc', 'ip_epc_dir',
            'ip_e1_value', 'ip_e2_value',
            'ip_last_dir',

            # Round two fields
            'pph', 'ppl',
            'icph', 'icpl',
            'refh', 'refl',
            'pp_reu', 'pp_red',
            'pp_reu_flag', 'pp_red_flag',
            'pp_last_dir',
        ]

        for directory in [self.output_child_dir, self.output_parent_dir, self.test_dir]:
            os.makedirs(directory, exist_ok=True)
            print(f"📁 Checked/Created directory: {directory}")

        for directory in [self.output_child_dir, self.output_parent_dir, self.test_dir]:
            try:
                os.makedirs(directory, exist_ok=True)
                logging.info(f"✅ Verified or created directory: {directory}")

                # Check write permissions
                if not os.access(directory, os.W_OK):
                    raise PermissionError(f"❌ No write access to: {directory}")
            except Exception as e:
                logging.error(f"❌ Failed to create/access directory {directory}: {e}")
                raise

        # Log directory paths
        #logging.info(f"📂 Input Directory: {self.input_dir}")
        #logging.info(f"📂 Output Child Directory: {self.output_child_dir}")
        #logging.info(f"📂 Output Parent Directory: {self.output_parent_dir}")
        #logging.info(f"📂 Test Directory: {self.test_dir}")

        # Ensure directories exist
        for directory in [self.output_child_dir, self.output_parent_dir, self.test_dir]:
            os.makedirs(directory, exist_ok=True)
            logging.info(f"✅ Verified or created directory: {directory}")

        for directory in [self.output_child_dir, self.output_parent_dir, self.test_dir]:
            if not os.access(directory, os.W_OK):
                raise PermissionError(f"❌ No write access to: {directory}")

        # All validation for input data--maybe ensure parsed child_period matches setting.
    def validate_input_data(self, df):
        """Validate input data according to spec requirements"""
        #if df.empty:
        #    logging.error("❗ Input data is empty")
        #raise ValueError("Input data is empty")

        required_columns = ['open', 'high', 'low', 'close', 'pph', 'ppl', 'ppc']
        missing_cols = [col for col in required_columns if col not in df.columns]
        if missing_cols:
            logging.error(f"❌ Missing required columns: {missing_cols}")
            raise ValueError(f"Missing columns in input data: {missing_cols}")

        # Check if 'high' and other price fields are numeric
        for col in ['open', 'high', 'low', 'close', 'pph', 'ppl', 'ppc']:
            if not pd.api.types.is_numeric_dtype(df[col]):
                logging.error(f"❌ Column '{col}' contains non-numeric data.")
                raise TypeError(f"Column '{col}' must contain numeric data.")

        conditions = [
            (df['open'] > 0, "open must be greater than 0"),
            (df['high'] > 0, "high must be greater than 0"),
            (df['low'] > 0, "low must be greater than 0"),
            (df['close'] > 0, "close must be greater than 0"),
            (df['low'] <= df['close'], "low must be less than or equal to close"),
            (df['close'] <= df['high'], "close must be less than or equal to high"),
            (df['low'] <= df['open'], "low must be less than or equal to open"),
            (df['open'] <= df['high'], "open must be less than or equal to high"),
            ((df['high'] - df['low']) != 0, "high minus low cannot be zero"),

            # Prior Parent Fields Validation
            (df['pph'] > 0, "prior parent high must be greater than 0"),
            (df['ppl'] > 0, "prior parent low must be greater than 0"),
            (df['ppc'] > 0, "close must be greater than 0"),
            (df['ppl'] <= df['ppc'], "low must be less than or equal to close"),
            (df['ppc'] <= df['pph'], "close must be less than or equal to high"),
        ]

        for condition, message in conditions:
            if not condition.all():
                logging.error(f"❌ Data validation failed: {message}")
                raise ValueError(f"Data validation failed: {message}")

        logging.info("✅ Input data passed all validation checks.")
        return True

    def safe_max(self, a, b):
        """Safely computes max while handling None values"""
        if a is None:
            return b
        if b is None:
            return a
        return max(a, b)

    def safe_min(self, a, b):
        """Safely computes min while handling None values"""
        if a is None:
            return b
        if b is None:
            return a
        return min(a, b)

    def safe_compare(self, a, b, op='gt'):
        """Safe comparison handling None values"""
        if a is None or b is None:
            return False
        if op == 'gt':
            return a > b
        elif op == 'lt':
            return a < b
        return a == b

    def safe_subtract(self, a, b):
        """Safe subtraction handling None values"""
        try:
            if a is None or b is None:
                return 0
            return a - b
        except Exception as e:
            logging.error(f"❗ Error during subtraction: {e}")
            return 0

    def process_round_one(self, data):

        """
        First Round - Establish intra-group values and prior parent expansions.
        Implements the relationship between intra-parent (IP) values, prior parent (PP) bounds, and prior child (PC) values.
        """
        print("🔍 Columns at start of Round 1:", data.columns.tolist())

        # Normalize column names to avoid issues with casing or whitespace
        data.columns = data.columns.str.strip().str.lower()

        # Check if 'high' column exists
        if 'high' not in data.columns:
            raise KeyError("❗ Column 'high' is missing in the input data.")

        # Check for empty DataFrame
        if data.empty:
            raise ValueError("❗ Input data is empty before Round 1 processing.")

        output_data = []

        # Initialize group state tracker with default values
        group_state = {
            "current_group": None,

            "current_group": None,
            "intra_group_h": None,
            "intra_group_l": None,
            "prior_gelo": None,
            "prior_gelc": None,
            "prior_gelh": None,
            "prior_gell": None,
            "prior_icph": None,  # Track previous row's icph
            "prior_icpl": None,   # Track previous row's icpl
            "pbh": None,  # Track previous row's high
            "pbl": None,   # Track previous row's low
            "pbc": None,   # Track previous row's close

            # Intra-Parent State Tracking
            "ip_state_dir": None,
            "ip_state_id": None,
            "ip_rpc_total": 0,

            "prior_ip_state_dir": None,
            "prior_ip_state_id": None,
            "prior_ip_rpc_total": 0,
            "prior_pp_state_dir": None,
            "prior_pp_state_id": None,
            "prior_pp_rpc_total": 0,

        }

        for index, row in data.iterrows():
            if str(row["parent"]) != str(group_state["current_group"]):
                group_state["current_group"] = row["parent"]
                group_state["intra_group_h"] = row["high"]
                group_state["intra_group_l"] = row["low"]
                group_state["gel_o"] = row["open"]
                group_state["prior_gelc"] = None
                group_state["prior_gelh"] = None
                group_state["prior_gell"] = None
                group_state["prior_icph"] = row["pph"]  # For sequence 1, use pph
                group_state["prior_icpl"] = row["ppl"]  # For sequence 1, use ppl
                group_state["pbc"] = None  # Add this line to initialize pbc

                # Initialize Prior Child with current row values
                #group_state["prior_open"] = row["open"]
                #group_state["prior_high"] = row["high"]
                #group_state["prior_low"] = row["low"]
                #group_state["prior_close"] = row["close"]

                # Initialize Prior Intra-Parent to None for new month
                #group_state["pip_open"] = None
                #group_state["pip_high"] = None
                #group_state["pip_low"] = None
                #group_state["pip_close"] = None

                # Initialize Intra-Parent with current row values
                #group_state["ip_open"] = row["open"]
                #group_state["ip_high"] = row["high"]
                #group_state["ip_low"] = row["low"]
                #group_state["ip_close"] = row["close"]

                # Reset state tracking for new month
                group_state["prior_ip_state_dir"] = None
                group_state["prior_ip_state_id"] = None
                group_state["prior_ip_rpc_total"] = None

                group_state["prior_pp_state_dir"] = None
                group_state["prior_pp_state_id"] = None
                group_state["prior_pp_rpc_total"] = None

                range_expansion_up = 0
                range_expansion_down = 0
                prior_percent_r = None
            else:
                # Calculate regular range expansions
                if (row["high"] > group_state["intra_group_h"]):
                    range_expansion_up = row["high"] - group_state["intra_group_h"]
                else:
                    range_expansion_up = 0

                if (row["low"] < group_state["intra_group_l"]):
                    range_expansion_down = group_state["intra_group_l"] - row["low"]
                else:
                    range_expansion_down = 0

            range = row["high"] - row["low"]
            chg_h = row["high"] - row["open"]
            chg_l = row["open"] - row["low"]
            chg_h_percent = ((row["high"] - row["open"]) / row["open"]) if row["open"] != 0 else 0
            chg_l_percent = ((row["open"] - row["low"]) / row["open"]) if row["open"] != 0 else 0
            body = max(row["open"], row["close"]) - min(row["open"], row["close"])
            gap = abs(row["open"] - group_state["pbc"]) if group_state["pbc"] is not None else None
            if group_state["pbh"] is not None and group_state["pbl"] is not None:
                if row["open"] > group_state["pbh"]:
                    island_gap = row["open"] - group_state["pbh"]
                elif row["open"] < group_state["pbl"]:
                   island_gap = row["low"] - row["open"]
                else:
                    island_gap = 0
            else:
               island_gap = 0

            # Update intra-group high and low: END OF ROW HIGH/LOW OF GEL
            group_state["intra_group_h"] = max(group_state["intra_group_h"], row["high"])
            group_state["intra_group_l"] = min(group_state["intra_group_l"], row["low"])

            # Calculate icph/icpl (using gel values AT OPEN against parent bounds)
            icph = max(group_state["intra_group_h"], row["pph"])
            icpl = min(group_state["intra_group_l"], row["ppl"])

            # Get refh/refl from prior state
            refh = group_state["prior_icph"]
            refl = group_state["prior_icpl"]

            # Calculate prior parent range expansions
            pp_reu = max(group_state["intra_group_h"] - refh, 0) if refh is not None else 0
            pp_red = max(refl - group_state["intra_group_l"], 0) if refl is not None else 0

            # Calculate prior parent range expansions
            bpb_reu = max(row["high"] - group_state["pbh"], 0) if group_state["pbh"] is not None else 0
            bpb_red = max(group_state["pbl"] - row["low"], 0) if group_state["pbl"] is not None else 0

            # Calculate prior_percent_r
            if group_state["prior_gelc"] is not None and (group_state["prior_gelh"] - group_state["prior_gell"]) != 0:
                prior_percent_r = (group_state["prior_gelc"] - group_state["prior_gell"]) / (
                    group_state["prior_gelh"] - group_state["prior_gell"]
                )
            else:
                prior_percent_r = None

            if group_state["prior_gelc"] is not None and group_state["prior_gelh"] - group_state["prior_gell"] != 0:
                gel_prior_range = group_state["prior_gelh"] - group_state["prior_gell"]
            else:
                gel_prior_range = None

            if group_state["pbc"] is not None and (group_state["pbh"] - group_state["pbl"]) != 0:
                bpb_pripr = (group_state["pbc"] - group_state["pbl"]) / (
                    group_state["pbh"] - group_state["pbl"]
                )
            else:
                bpb_pripr = None
            # Update prior values for next row
            #group_state["prior_gelo"] = row["gelo"]
            group_state["prior_gelc"] = row["close"]
            group_state["prior_gelh"] = group_state["intra_group_h"]
            group_state["prior_gell"] = group_state["intra_group_l"]
            group_state["prior_icph"] = icph
            group_state["prior_icpl"] = icpl
            group_state["pbc"] = row["close"]

            # Create processed row
            processed_row = row.to_dict()
            processed_row.update({
                "gel_o": group_state["gel_o"],
                "gel_h": group_state["intra_group_h"],
                "gel_l": group_state["intra_group_l"],
                "gel_reu_value": range_expansion_up,
                "gel_red_value": range_expansion_down,
                "icph": icph,
                "icpl": icpl,
                "refh": refh,
                "refl": refl,
                "pp_reu": pp_reu,
                "pp_red": pp_red,
                "gel_pripr": prior_percent_r,
                "gel_prirange": gel_prior_range,
                "bpb_reu_value": bpb_reu,
                "bpb_red_value": bpb_red,
                "pbh": group_state["pbh"],
                "pbl": group_state["pbl"],
                "pbc": group_state["pbc"],
                "bpb_pripr": bpb_pripr,
                "range": range,
                "chg_h": chg_h,
                "chg_l": chg_l,
                "chg_h_percent": chg_h_percent,
                "chg_l_percent": chg_l_percent,
                "body": body,
                "gap": gap,
                "island_gap": island_gap
            })
            output_data.append(processed_row)

            group_state["pbh"] = row["high"]
            group_state["pbl"] = row["low"]
            group_state["pbc"] = row["close"]

        return pd.DataFrame(output_data)

    def process_round_two(self, data):
        """
        Second Round - Range Expansion with Prior Child, Intra-Parent, and Prior Parent Relationships
        """
        print("🔍 Columns at start of Round 2:", data.columns.tolist())
        required_columns = ["high", "low", "close", "sequence", "pph", "ppl"]
        missing_cols = [col for col in required_columns if col not in data.columns]
        if missing_cols:
            raise KeyError(f"❗ Missing columns: {missing_cols}")
        # --- Prior Child (PC) Calculations ---

        #data["bpb_reu_value"] = data.apply(
        #   lambda row: self.safe_subtract(row["high"], row["prior_high"]) if self.safe_compare(row["high"], row["prior_high"], 'gt') else 0,
        #   axis=1
        #)
        #data["bpb_red_value"] = data.apply(
        #    lambda row: ((row["close"] - row["low"]) / (row["high"] - row["low"])) if (row["high"] - row["low"]) != 0 else 0
        #)
        #data["bpb_pri_percentr"] = data.apply(
        #    lambda row: ((row["prior_close"] - row["prior_low"]) / (row["prior_high"] - row["prior_low"]))
        #    if (row["prior_high"] - row["prior_low"]) != 0 and None not in (row["prior_close"], row["prior_low"], row["prior_high"]) else 0,
        #    axis=1
        #)
        #data["bpb_pri_percentr"] = data.apply(
        #    lambda row: ((row["close"] - row["low"]) / (row["high"] - row["low"])) if (row["high"] - row["low"]) != 0 else 0
        #)
        data["bpb_ce_percent"] = data.apply(
            lambda row: None if pd.isna(row["bpb_pripr"]) else
                (1 - row["bpb_pripr"] if row["bpb_pripr"] >= 0.5 else row["bpb_pripr"]),
            axis=1
        )
        data["bpb_epc"] = data["bpb_ce_percent"].apply(
        lambda x: None if pd.isna(x) else max(1, min(5, math.ceil(x / 0.1)))
        )
        data["bpb_epc_dir"] = np.where(
            data["sequence"] == 1,
            None,
            np.where(data["bpb_pripr"] >= 0.5, "U", "D")
        )
        data["bpb_e1_value"] = data.apply(
            lambda row: row["bpb_reu_value"] if row["bpb_pripr"] >= 0.5 else row["bpb_red_value"],
            axis=1
        )
        data["bpb_e2_value"] = data.apply(
            lambda row: row["bpb_reu_value"] if row["bpb_pripr"] < 0.5 else row["bpb_red_value"],
            axis=1
        )
        data["bpb_red_flag"] = data["bpb_red_value"] > 0
        data["bpb_reu_flag"] = data["bpb_reu_value"] > 0
        data["bpb_re_flag"] = data["bpb_reu_flag"] | data["bpb_red_flag"]
        data["bpb_twoway_flag"] = data["bpb_reu_flag"] & data["bpb_red_flag"]

        # --- Intra-Parent (IP) Calculations ---
        #data["ip_reu_value"] = data.apply(
        #    lambda row: ((row["close"] - row["low"]) / (row["high"] - row["low"])) if (row["high"] - row["low"]) != 0 else 0
        #)
        #data["ip_red_value"] = data.apply(
        #    lambda row: ((row["close"] - row["low"]) / (row["high"] - row["low"])) if (row["high"] - row["low"]) != 0 else 0
        #)
        data["gel_reu_flag"] = data.apply(
            lambda row: None if row["sequence"] == 1 else (True if row["gel_reu_value"] > 0 else False),
            axis=1
        )

        data["gel_red_flag"] = data.apply(
            lambda row: None if row["sequence"] == 1 else (True if row["gel_red_value"] > 0 else False),
            axis=1
        )

        #data["ip_pri_percentr"] = data.apply(
        #    lambda row: ((row["pip_close"] - row["pip_low"]) / (row["pip_high"] - row["pip_low"]))
        #    if (row["pip_high"] - row["pip_low"]) != 0 else 0,
        #    axis=1
        #)
        #data["ip_pri_percentr"] = data.apply(
        #    lambda row: 0 if pd.isna(row["high"]) or pd.isna(row["low"]) or (row["high"] - row["low"]) == 0
        #    else (row["close"] - row["low"]) / (row["high"] - row["low"]),
        #)
        data["ip_ce_percent"] = data.apply(
            lambda row: row["gel_pripr"] if row["gel_pripr"] < 0.5 else row["gel_pripr"],
            axis = 1
        )
        data["ip_epc"] = data["ip_ce_percent"].apply(
            lambda x: None if pd.isna(x) else max(1, min(5, math.ceil(x / 0.1)))

        )
        data["ip_epc_dir"] = data.apply(
            lambda row: None if row["sequence"] == 1 else ("U" if row["gel_pripr"] >= 0.5 else "D"),
            axis=1
        )
        data["ip_e1_value"] = data.apply(
            lambda row: row["gel_reu_value"] if row["gel_pripr"] >= 0.5 else row["gel_red_value"],
            axis=1
        )
        data["ip_e2_value"] = data.apply(
            lambda row: row["gel_reu_value"] if row["gel_pripr"] < 0.5 else row["gel_red_value"],
            axis=1
        )
        if 'high' not in data.columns:
            raise KeyError("❗ 'high' column is missing in process_round_two")
        #data["gel_reu_flag"] = data["gel_reu_value"] > 0
        #data["gel_red_flag"] = data["gel_red_value"] > 0
        data["ip_re_flag"] = data["gel_reu_flag"] | data["gel_red_flag"]
        data["ip_twoway_flag"] = data["gel_reu_flag"] & data["gel_red_flag"]
        data["ip_twoway_fre_dir"] = data.apply(
            lambda row: None if row["sequence"] == 1 else
                (row["ip_epc_dir"] if row["ip_twoway_flag"] == True else "N"),
            axis=1
        )

        data["ip_oneway_fre_dir"] = data.apply(
            lambda row: None if row["sequence"] == 1 else
                (("D" if row["gel_red_flag"] == True else "N") if row["gel_reu_flag"] == False else "U"),
            axis=1
        )

        data["ip_last_dir"] = data.apply(
            lambda row: None if row["sequence"] == 1 else
                ("D" if bool(row["ip_twoway_flag"]) and row["ip_twoway_fre_dir"] == "U" else
               "U" if bool(row["ip_twoway_flag"]) and row["ip_twoway_fre_dir"] == "D" else
               "U" if bool(row["gel_reu_flag"]) else
               "D" if bool(row["gel_red_flag"]) else
               "N"),
            axis=1
        )
        # Prior parent range expansions
        data["pp_reu_flag"] = data.apply(
            lambda row: None if row["sequence"] == 1 else (True if row["pp_reu"] > 0 else False),
            axis=1
        )

        data["pp_red_flag"] = data.apply(
            lambda row: None if row["sequence"] == 1 else (True if row["pp_red"] > 0 else False),
            axis=1
        )
        data["pp_pri_percentr"] = data.apply(
            lambda row: None if pd.isna(row["ppc"]) or (row["pph"] - row["ppl"]) == 0
            else (row["ppc"] - row["ppl"]) / (row["pph"] - row["ppl"]),
            axis = 1
        )
        data["pp_ce_percent"] = data.apply(
            lambda row: None if row["sequence"] == 1 else
                (1 - row["pp_pri_percentr"] if row["pp_pri_percentr"] >= 0.5 else row["pp_pri_percentr"]),
            axis=1
        )
        data["pp_epc"] = data["pp_ce_percent"].apply(
            lambda x: None if pd.isna(x) else max(1, min(5, math.ceil(x / 0.1)))
        )
        data["pp_epc_dir"] = data.apply(
            lambda row: None if row["sequence"] == 1 else ("U" if row["pp_pri_percentr"] >= 0.5 else "D"),
            axis=1
        )
        data["pp_e1_value"] = data.apply(
            lambda row: row["pp_reu"] if row["pp_pri_percentr"] >= 0.5 else row["pp_red"],
            axis=1
        )
        data["pp_e2_value"] = data.apply(
            lambda row: row["pp_reu"] if row["pp_pri_percentr"] < 0.5 else row["pp_red"],
            axis=1
        )
        data["pp_re_flag"] = data["pp_reu_flag"] | data["pp_red_flag"]
        data["pp_twoway_flag"] = data["pp_reu_flag"] & data["pp_red_flag"]
        data["pp_twoway_fre_dir"] = data.apply(
            lambda row: None if row["sequence"] == 1 else
                (row["pp_epc_dir"] if row["pp_twoway_flag"] == True else "N"),
            axis=1
        )

        data["pp_oneway_fre_dir"] = data.apply(
            lambda row: None if row["sequence"] == 1 else
                (("D" if row["pp_red_flag"] == True else "N") if row["pp_reu_flag"] == False else "U"),
            axis=1
        )

        data["pp_last_dir"] = data.apply(
            lambda row:
                ("D" if row["pp_twoway_flag"] and row["pp_twoway_fre_dir"] == "U" else
                 "U" if row["pp_twoway_flag"] and row["pp_twoway_fre_dir"] == "D" else
                 "U" if row["pp_reu_flag"] else
                 "D" if row["pp_red_flag"] else
                 "N"),
            axis=1
        )
        #data["pp_co_flag"] = data.apply(
        #    lambda row: None if row["sequence"] == 1 else
        #        (False if row["low"] > row["pph"] else False if row["high"] < row["ppl"] else True),
        #    axis=1
        #)
        #data["pp_noco_dir"] = data.apply(
        #    lambda row: None if row["sequence"] == 1 else
        #        (None if row["pp_co_flag"] == True else "A" if row["low"] > row["pph"] else "B"),
        #    axis=1
        #)

        return data

    def process_round_three_ip(self, data):
        """Third Round - RPC States"""
        state_tracker = {
            "prior_ip_state_dir": None,
            "prior_ip_last_dir": None,
            "prior_ip_rpc_total": 0,
        }

        for index, row in data.iterrows():
            try:
                sequence = row["sequence"]
                ip_last_dir = row["ip_last_dir"]
                ip_twoway_flag = row["ip_twoway_flag"]

                if sequence == 1:
                    ip_state_dir = "N"
                    ip_rpc_this_row = 0
                    ip_rpc_total = 0
                    ip_state_id = f"{row['parent']}_{sequence}_N"
                else:
                    # Calculate gel_dir_state
                    ip_dir_state = ip_last_dir if ip_last_dir != "N" else state_tracker["prior_ip_state_dir"]

                    # Calculate gel_rpc
                    if sequence == 2 and ip_twoway_flag:
                        ip_rpc_this_row = 2
                    elif ip_last_dir == "N":
                        ip_rpc_this_row = 0
                    elif ip_last_dir == state_tracker["prior_ip_last_dir"] and ip_twoway_flag:
                        ip_rpc_this_row = 2
                    elif ip_last_dir != state_tracker["prior_ip_state_dir"] and ip_last_dir != "N":
                        ip_rpc_this_row = 1
                    else:
                        ip_rpc_this_row = 0

                    # Calculate ip_rpc_total safely
                    ip_rpc_total = state_tracker["prior_ip_rpc_total"] + ip_rpc_this_row if state_tracker["prior_ip_rpc_total"] is not None else ip_rpc_this_row

                    # Generate ip_state_id if new rpc occurs
                    ip_state_id = f"{row['parent']}_{sequence}_{ip_last_dir}" if ip_rpc_this_row == 1 else state_tracker["prior_ip_state_dir"]

                # Update DataFrame
                data.loc[index, "ip_state_dir"] = ip_state_dir
                data.loc[index, "ip_rpc_this_row"] = ip_rpc_this_row
                data.loc[index, "ip_rpc_total"] = ip_rpc_total
                data.loc[index, "ip_state_id"] = ip_state_id


                # Update state tracker
                state_tracker.update({
                    "prior_ip_state_dir": ip_state_dir,
                    "prior_ip_last_dir": ip_last_dir,
                    "prior_ip_rpc_total": ip_rpc_total,
                })

            except Exception as e:
                logging.error(f"Error processing row {index} in round three ip: {e}")
                print(f"Error details - state_tracker: {state_tracker}")
                raise

        return data

    def process_round_three_pp(self, data):
        """Third Round - RPC States for Child/Prior Parent (PP)"""
        state_tracker = {
            "prior_pp_state_dir": None,
            "prior_pp_last_dir": None,
            "prior_pp_rpc_total": 0,
        }

        for index, row in data.iterrows():
            sequence = row["sequence"]
            pp_last_dir = row["pp_last_dir"]
            pp_twoway_flag = row["pp_twoway_flag"]

            if sequence == 1:
                pp_state_dir = "N"
                pp_rpc_this_row = 0  # Represents RPC state change for this specific row
                pp_rpc_total = 0
                pp_state_id = f"{row['parent']}_{sequence}_N"
            else:
                # Calculate pp_dir_state
                pp_dir_state = pp_last_dir if pp_last_dir != "N" else state_tracker["prior_pp_state_dir"]

                # Calculate pp_rpc_this_row
                if sequence == 2 and pp_twoway_flag:
                    pp_rpc_this_row = 2
                elif pp_last_dir == "N":
                    pp_rpc_this_row = 0
                elif pp_last_dir == state_tracker["prior_pp_last_dir"] and pp_twoway_flag:
                    pp_rpc_this_row = 2
                elif pp_last_dir != state_tracker["prior_pp_state_dir"] and pp_last_dir != "N":
                    pp_rpc_this_row = 1
                else:
                    pp_rpc_this_row = 0

                # Calculate pp_rpc_total
                pp_rpc_total = state_tracker["prior_pp_rpc_total"] + pp_rpc_this_row

                # Generate pp_state_id if new rpc occurs
                if pp_rpc_this_row == 1:
                    pp_state_id = f"{row['parent']}_{sequence}_{pp_last_dir}"
                else:
                    pp_state_id = state_tracker["prior_pp_state_dir"]

            # Update state tracker
            state_tracker.update({
                "prior_pp_state_dir": pp_state_dir,
                "prior_pp_last_dir": pp_last_dir,
                "prior_pp_rpc_total": pp_rpc_total
            })

            # Update DataFrame
            data.at[index, "pp_state_dir"] = pp_state_dir
            data.at[index, "pp_rpc_this_row"] = pp_rpc_this_row
            data.at[index, "pp_rpc_total"] = pp_rpc_total
            data.at[index, "pp_state_id"] = pp_state_id

        return data

    def generate_summary(self, data, ticker):
        """Generate summary data for parent periods"""
        summary_data = []

        for parent, group in data.groupby("parent"):
            lookup_date = group["parent"].iloc[0]
            duration = len(group)
            parent_high = group["gel_h"].max()
            parent_low = group["gel_l"].min()
            bar_of_h = group.loc[group["gel_h"].idxmax(), "sequence"]
            bar_of_l = group.loc[group["gel_l"].idxmin(), "sequence"]
            bpb_reu_count = group["bpb_reu_flag"].sum()
            bpb_red_count = group["bpb_red_flag"].sum()
            bpb_reu_max = group["bpb_reu_value"].max()
            bpb_red_max = group["bpb_red_value"].max()
            bpb_re_count = group["bpb_re_flag"].sum()
            bpb_no_re_count = duration - group["bpb_re_flag"].sum()
            bpb_twoway_count = group["bpb_twoway_flag"].sum()
            range_max = group["range"].max()
            range_avg = group["range"].mean()
            bpb_re_count = group["bpb_re_flag"].sum()
            bpb_no_re_count = duration - group["bpb_re_flag"].sum()
            #bpb_twoway_count = group["bpb_twoway_flag"].sum()
            A1 = min(bar_of_h, bar_of_l)
            A2 = max(bar_of_h, bar_of_l)
            gel_reu_count = group["gel_reu_flag"].sum()
            gel_red_count = group["gel_red_flag"].sum()
            gel_reu_first = group.loc[group["gel_reu_flag"] == True, "sequence"].min() if gel_reu_count > 0 else None
            gel_reu_last = group.loc[group["gel_reu_flag"] == True, "sequence"].max() if gel_reu_count > 0 else None
            gel_red_first = group.loc[group["gel_red_flag"] == True, "sequence"].min() if gel_red_count > 0 else None
            gel_red_last = group.loc[group["gel_red_flag"] == True, "sequence"].max() if gel_red_count > 0 else None

            #gel_rpc_total = group["gel_rpc"].sum()

            #pp_reu_count = group["pp_reu_flag"].sum()
            #pp_red_count = group["pp_red_flag"].sum()
            #pp_reu_first = group.loc[group["pp_reu_flag"] == True, "sequence"].min() if pp_reu_count > 0 else None
            #pp_reu_last = group.loc[group["pp_reu_flag"] == True, "sequence"].max() if pp_reu_count > 0 else None
            #pp_red_first = group.loc[group["pp_red_flag"] == True, "sequence"].min() if pp_red_count > 0 else None
            #pp_red_last = group.loc[group["pp_red_flag"] == True, "sequence"].max() if pp_red_count > 0 else None

            summary_data.append({
                "ticker": ticker,
                "parent": lookup_date,
                "duration": duration,
                "child_period": self.child_period,
                "parent_period": self.parent_period,
                "range_max": range_max,
                "range_avg": range_avg,
                "bpb_reu_count": bpb_reu_count,
                "bpb_red_count": bpb_red_count,
                "bpb_reu_max": bpb_reu_max,
                "bpb_red_max": bpb_red_max,
                "bpb_re_count": bpb_re_count,
                "bpb_no_re_count": bpb_no_re_count,
                "bpb_twoway_count": bpb_twoway_count,
                "parent_high": parent_high,
                "parent_low": parent_low,
                "bar_of_h": bar_of_h,
                "bar_of_l": bar_of_l,
                "A1": min(bar_of_h, bar_of_l),
                "A2": max(bar_of_h, bar_of_l),
                "gel_reu_count": gel_reu_count,
                "gel_red_count": gel_red_count,
                "gel_reu_first": gel_reu_first,
                "gel_reu_last": gel_reu_last,
                "gel_red_first": gel_red_first,
                "gel_red_last": gel_red_last,
                #"pp_reu_count": pp_reu_count,
                #"pp_red_count": pp_red_count,
                #"pp_reu_first": pp_reu_first,
                #"pp_reu_last": pp_reu_last,
                #"pp_red_first": pp_red_first,
                #"pp_red_last": pp_red_last,
                #"gel_rpc_total": gel_rpc_total,
                "create_date": datetime.now().date(),
                "create_time": datetime.now().time(),
                "jobname": self.jobname
            })

        return pd.DataFrame(summary_data)

    def export_test_file(self, data, ticker):
        """Export all fields after round three processing to a test file"""
        # Assign ticker to the data
        # ✅ Add this to verify the export process
        print(f"Attempting to export test file for {ticker}")

        # ⚠️ Check if DataFrame is empty
        if data.empty:
            print("⚠️ Data is empty, nothing to export.")
            return data  # Stop here if data is empty
        else:
            print(f"Data has {len(data)} rows. Proceeding with export.")

        data['ticker'] = ticker

        # Updated field list to reflect the latest spec
        columns = [
            # Original input fields
            'ticker', 'date', 'parent', 'sequence', 'open', 'high', 'low', 'close', 'range',
            'chg_h', 'chg_l', 'chg_h_percent', 'chg_l_percent',

            # Prior Child (PC) Fields
            'bpb_red_value', 'bpb_red_flag',
            'bpb_pripr', 'bpb_ce_percent', 'bpb_epc', 'bpb_epc_dir',

            # Intra-Parent (IP) Fields
            'gel_o', 'gel_h', 'gel_l', 'close',
            'gel_reu_value', 'gel_red_value', 'gel_reu_flag', 'gel_red_flag',
            'gel_pripr', 'ip_ce_percent', 'ip_epc', 'ip_epc_dir',
            'ip_e1_value', 'ip_e2_value', 'ip_re_flag', 'ip_twoway_flag',
            'ip_state_dir', 'ip_rpc_this_row', 'ip_rpc_total', 'ip_state_id',

        ]

        # ✅ Ensure test directory exists
        if not os.path.exists(self.test_dir):
            try:
                os.makedirs(self.test_dir, exist_ok=True)
                print(f"📂 Created missing test directory: {self.test_dir}")
            except Exception as e:
                print(f"❌ Failed to create test directory {self.test_dir}: {str(e)}")
                return data

       # ✅ Check if the directory is writable
        if not os.access(self.test_dir, os.W_OK):
            print(f"❌ No write access to: {self.test_dir}")
            return data

        # ✅ Ensure all columns exist in the DataFrame
        missing_columns = [col for col in columns if col not in data.columns]
        if missing_columns:
            print(f"⚠️ Missing columns in data, cannot export: {missing_columns}")
            return data

        # Create filename with timestamp for uniqueness
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        output_file = os.path.join(self.test_dir, f"{ticker}_test_{timestamp}.csv")

        # Export to CSV with error handling
        try:
            data.to_csv(output_file, columns=columns, index=False)
            print(f"Exported test file to: {output_file}")
        except Exception as e:
            print(f"Failed to export test file for {ticker}: {str(e)}")

        return data

    def process_file(self, filename):
        """Process a single input file through all rounds."""
        try:
            # --- Validate Filename Format ---
            if not filename.endswith(f"_{self.child_period}.csv"):
                raise ValueError(f"Invalid filename format: {filename}. Expected format: {{TICKER}}_{self.child_period}.csv")

            # --- Extract Ticker ---
            ticker = filename.split('_')[0]

            logging.info(f"📥 Reading file: {filename}")

            print(f"Reading file: {filename}")
            # --- Read Input File ---
            input_path = os.path.join(self.input_dir, filename)
            try:
                data = pd.read_csv(input_path)
                logging.info(f"✅ Successfully loaded input file: {input_path}")
            except FileNotFoundError:
                logging.error(f"❌ File not found: {input_path}")
                raise FileNotFoundError(f"Input file not found: {input_path}")
            except pd.errors.EmptyDataError:
                logging.error(f"❗ File is empty: {input_path}")
                raise ValueError(f"Input file is empty: {input_path}")
            except pd.errors.ParserError as e:
                logging.error(f"❗ Error parsing file {input_path}: {e}")
                raise ValueError(f"Error parsing file: {input_path}")
            except Exception as e:
                logging.error(f"❗ Unexpected error reading file {input_path}: {e}")
                raise

            # --- Validate Input Data ---
            self.validate_input_data(data)
            print("✅ Data validation passed")

            # --- Add Required Metadata Columns ---
            data['child_period'] = self.child_period
            data['parent_period'] = self.parent_period
            data['jobname'] = self.jobname

            # --- Process Through All Rounds ---
            data = self.process_round_one(data)
            print("✅ Completed Round 1")
            print(f"🔎 After Round 1 - Data shape: {data.shape}")
            data = self.process_round_two(data)
            print("✅ Completed Round 2")
            print(f"🔎 After Round 2 - Data shape: {data.shape}")
            data = self.process_round_three_ip(data)
            print("✅ Completed Round 3ip")
            data = self.process_round_three_pp(data)
            print("✅ Completed Round 3pp")
            print(f"🔎 After Round 3bb - Data shape: {data.shape}")

            # --- Generate Summary ---
            self.validate_input_data(data)
            logging.info("✅ Data validation passed")

            # --- Add Required Metadata Columns ---
            data['child_period'] = self.child_period
            data['parent_period'] = self.parent_period
            data['jobname'] = self.jobname

            # --- Process Through All Rounds ---
            data = self.process_round_one(data)
            logging.info("✅ Completed Round 1")

            data = self.process_round_two(data)
            logging.info("✅ Completed Round 2")

            data = self.process_round_three_ip(data)
            logging.info("✅ Completed Round 3")

            data = self.process_round_three_pp(data)
            logging.info("✅ Completed Round 3")

            # --- Export Test File ---
            self.export_test_file(data, ticker)
            logging.info("✅ Exported test file")

            # --- Generate Summary ---
            summary_df = self.generate_summary(data, ticker)
            logging.info("✅ Generated summary data")

            # --- Export Test File ---
            self.export_test_file(data, ticker)
            logging.info("✅ Exported test file")

            # --- Export Child Data ---
            child_filename = f"{ticker}_{self.child_period}_processed.csv"
            child_path = os.path.join(self.output_child_dir, child_filename)

            # Check write permission
            if not os.access(self.output_child_dir, os.W_OK):
                logging.error(f"❌ No write access to: {self.output_child_dir}")
                return None, None

            data.to_csv(child_path, columns=self.export_columns, index=False)
            logging.info(f"✅ Exported child data to: {child_path}")

            # --- Export Summary Data ---
            summary_filename = f"{ticker}_{self.parent_period}_summary.csv"
            summary_path = os.path.join(self.output_parent_dir, summary_filename)

            if not os.access(self.output_parent_dir, os.W_OK):
                logging.error(f"❌ No write access to: {self.output_parent_dir}")
                return None, None

            summary_df.to_csv(summary_path, index=False)
            logging.info(f"✅ Exported parent summary to: {summary_path}")

            return data, summary_df

        except Exception as e:
            logging.error(f"❗ Error processing {filename}: {str(e)}")
            return None, None

    def process_all_files(self):
        """Process all eligible files in the input directory with progress tracking and robust error handling."""

        # --- Validate Input Directory ---
        if not os.path.exists(self.input_dir):
            raise FileNotFoundError(f"❌ Input directory {self.input_dir} does not exist.")

        # --- Check Write Permissions for Output Directories ---
        for directory in [self.output_child_dir, self.output_parent_dir, self.test_dir]:
            if not os.access(directory, os.W_OK):
                raise PermissionError(f"❌ No write access to: {directory}")

        results = {}
        processed_files = 0
        errors = []

        # --- List Eligible Files ---
        eligible_files = [f for f in os.listdir(self.input_dir) if f.endswith(f"_{self.child_period}.csv")]
        print(f"📂 Eligible Files: {eligible_files}")  # Debug print

        if not eligible_files:
            logging.warning(f"⚠️ No eligible files found with pattern *_{self.child_period}.csv")
            return results

        logging.info(f"📈 Found {len(eligible_files)} files to process.\n")

        # --- Track Total Processing Time ---
        total_start_time = time.time()

        # --- Process Each File with Progress Bar ---
        for idx, filename in enumerate(tqdm(eligible_files, desc="🚀 Processing Files", unit="file"), start=1):
            file_start_time = time.time()
            try:
                logging.info(f"\n📄 [{idx}/{len(eligible_files)}] Processing {filename}...")

                # --- Process the file ---
                data, summary_df = self.process_file(filename)

                if data is None or data.empty:
                    logging.warning(f"⚠️ Skipping {filename}: No data to process.")
                    continue

                # --- Store Results ---
                results[filename] = {
                    'status': 'success',
                    'data': data,
                    'summary': summary_df,
                    'error': None,
                    'processing_time_sec': round(time.time() - file_start_time, 2)
                }

                processed_files += 1
                logging.info(f"✅ Successfully processed {filename} in {results[filename]['processing_time_sec']} seconds.")

            except Exception as e:
                error_msg = f"❗ Error processing {filename}: {str(e)}"
                logging.error(error_msg)
                errors.append(error_msg)
                results[filename] = {
                    'status': 'error',
                    'data': None,
                    'summary': None,
                    'error': str(e)
                }

        # --- Final Processing Summary ---
        total_elapsed_time = round(time.time() - total_start_time, 2)
        logging.info(f"\n🎉 Processing Complete: {processed_files}/{len(eligible_files)} files successfully processed in {total_elapsed_time} seconds.")

        if errors:
            logging.warning("\n⚠️ Errors encountered during processing:")
            for error in errors:
                logging.warning(f"- {error}")

        return results

# 🔥 Main Execution Block (PLACE THIS AT THE END)
if __name__ == "__main__":
    print("🚀 Starting GelSetProcessor...")

    # Initialize the processor
    processor = GelSetProcessor()

    # Start processing all files
    results = processor.process_all_files()

    logging.info("✅ Processing complete!")

🚀 Starting GelSetProcessor...
📁 Checked/Created directory: /content/output_child
📁 Checked/Created directory: /content/output_parent
📁 Checked/Created directory: /content/test
📂 Eligible Files: ['BAC_D.csv']


🚀 Processing Files:   0%|          | 0/1 [00:00<?, ?file/s]

Reading file: BAC_D.csv
✅ Data validation passed
🔍 Columns at start of Round 1: ['date', 'open', 'high', 'low', 'close', 'parent', 'sequence', 'pph', 'ppl', 'ppc', 'volume', 'child_period', 'parent_period', 'jobname']
✅ Completed Round 1
🔎 After Round 1 - Data shape: (7782, 41)
🔍 Columns at start of Round 2: ['date', 'open', 'high', 'low', 'close', 'parent', 'sequence', 'pph', 'ppl', 'ppc', 'volume', 'child_period', 'parent_period', 'jobname', 'gel_o', 'gel_h', 'gel_l', 'gel_reu_value', 'gel_red_value', 'icph', 'icpl', 'refh', 'refl', 'pp_reu', 'pp_red', 'gel_pripr', 'gel_prirange', 'bpb_reu_value', 'bpb_red_value', 'pbh', 'pbl', 'pbc', 'bpb_pripr', 'range', 'chg_h', 'chg_l', 'chg_h_percent', 'chg_l_percent', 'body', 'gap', 'island_gap']
✅ Completed Round 2
🔎 After Round 2 - Data shape: (7782, 75)
✅ Completed Round 3ip
✅ Completed Round 3pp
🔎 After Round 3bb - Data shape: (7782, 83)
🔍 Columns at start of Round 1: ['date', 'open', 'high', 'low', 'close', 'parent', 'sequence', 'pph', 'pp

🚀 Processing Files: 100%|██████████| 1/1 [00:34<00:00, 34.55s/file]
